In [1]:
import pandas as pd
full_df = pd.read_csv("../Data/JoinedData_6_26_2024__NotParcled.csv")

In [2]:
df = full_df.copy(deep=True)#pd.read_csv("JoinedData_2_14_2024.csv")

In [3]:
selected_columns = df.columns[df.columns.str.contains('age') & df.columns.str.contains('C')]

In [4]:
LatentSlopeWeights_Part1 = df[selected_columns].mean().round(6)

In [5]:
FullTrajecoryAgeDiff = LatentSlopeWeights_Part1['C13xactage']-LatentSlopeWeights_Part1['C01xactage']

In [6]:
MiddleOfFullTrajectoryAge = LatentSlopeWeights_Part1['C01xactage'] + FullTrajecoryAgeDiff/2

In [7]:
Weights = LatentSlopeWeights_Part1 - MiddleOfFullTrajectoryAge 
WEIGHTS = Weights/10
WEIGHTS.round(3)

C01xactage   -0.764
C02xactage   -0.666
C03xactage   -0.569
C04xactage   -0.470
C05xactage   -0.374
C06xactage   -0.270
C07xactage   -0.170
C08xactage   -0.077
C09xactage    0.025
C10xactage    0.136
C11xactage    0.324
C12xactage    0.468
C13xactage    0.764
dtype: float64

In [22]:
WeightsForFullParentTrajectories = WEIGHTS.round(3).to_dict()
NewKeys = ['W1','W2','W3','W4','W5','W6','W7','W8','W9','W10','W11','W12','W13']
WeightsForFullParentTrajectories = dict(zip(NewKeys, WeightsForFullParentTrajectories.values()))

In [23]:
WeightsForFullParentTrajectories

{'W1': -0.764,
 'W2': -0.666,
 'W3': -0.569,
 'W4': -0.47,
 'W5': -0.374,
 'W6': -0.27,
 'W7': -0.17,
 'W8': -0.077,
 'W9': 0.025,
 'W10': 0.136,
 'W11': 0.324,
 'W12': 0.468,
 'W13': 0.764}

In [11]:
ChildTrajecoryAgeDiff = LatentSlopeWeights_Part1['C13xactage']-LatentSlopeWeights_Part1['C05xactage']

In [12]:
ChildWeightsFromW5 = LatentSlopeWeights_Part1- ( LatentSlopeWeights_Part1['C05xactage'] + ChildTrajecoryAgeDiff/2)

In [24]:
ChildWeightsFromW5_By10= ChildWeightsFromW5/10
WeightsBasedOnFullChildTrajectories = ChildWeightsFromW5_By10.round(3).to_dict()
WeightsBasedOnFullChildTrajectories = dict(zip(NewKeys,WeightsBasedOnFullChildTrajectories.values()))

In [25]:
WeightsBasedOnFullChildTrajectories

{'W1': -0.958,
 'W2': -0.86,
 'W3': -0.764,
 'W4': -0.665,
 'W5': -0.569,
 'W6': -0.465,
 'W7': -0.364,
 'W8': -0.272,
 'W9': -0.169,
 'W10': -0.059,
 'W11': 0.13,
 'W12': 0.274,
 'W13': 0.569}

In [16]:
def GenerateLatentSlopeString(Person,FullOrOverlap,Domain):
    if FullOrOverlap == 'Full':
        
        if Person == 'Mom':
             Waves = [1,3,5,7,10,11,12,13]
        elif Person == 'Dad':
             Waves = [1,3,5,7,11,12,13]

        elif Person == 'Child':
            return GenerateLatentSlopeString(Person,'Overlap',Domain) #just because it's easier to classify the child "full" trajectory in with the overlap models.
            
        Weights =  WeightsForFullParentTrajectories
        
    elif FullOrOverlap == 'Overlap':
        
        if Person == 'Mom' or Person == 'Child':
            Waves = [5,7,10,11,12,13]
        elif Person == "Dad":
            Waves = [5,7,11,12,13]
      
        Weights = WeightsBasedOnFullChildTrajectories

        
    LatentSlopeFactorStringStart = Person+Domain+'Slope =~ '
    LatentSlopeFactorRHBits = []
    for W in Waves:
        WaveMarker =  'W' + str(W)
        Weight = Weights[WaveMarker]
        LatentSlopeFactorRHBits.append(str(Weight)+'*'+ WaveMarker+Domain+'_'+Person)
    return LatentSlopeFactorStringStart + '+'.join(LatentSlopeFactorRHBits)

In [17]:
GenerateLatentSlopeString('Child','Full','Extraversion')

KeyError: 'W5'

In [72]:
GenerateLatentSlopeString('Mom','Full','Extraversion')

'MomExtraversionSlope =~ -0.764*W1Extraversion_Mom+-0.569*W3Extraversion_Mom+-0.374*W5Extraversion_Mom+-0.17*W7Extraversion_Mom+0.136*W10Extraversion_Mom+0.324*W11Extraversion_Mom+0.468*W12Extraversion_Mom+0.764*W13Extraversion_Mom'

In [15]:
GenerateLatentSlopeString('Dad','Full','Extraversion')

NameError: name 'GenerateLatentSlopeString' is not defined

In [78]:
GenerateLatentSlopeString('Child','Full','Agreeableness')

'ChildAgreeablenessSlope =~ -0.569*W5Agreeableness_Child+-0.364*W7Agreeableness_Child+-0.059*W10Agreeableness_Child+0.13*W11Agreeableness_Child+0.274*W12Agreeableness_Child+0.569*W13Agreeableness_Child'

In [85]:
GenerateLatentSlopeString('Dad','Full','Agreeableness')

'DadAgreeablenessSlope =~ -0.764*W1Agreeableness_Dad+-0.569*W3Agreeableness_Dad+-0.374*W5Agreeableness_Dad+-0.17*W7Agreeableness_Dad+0.324*W11Agreeableness_Dad+0.468*W12Agreeableness_Dad+0.764*W13Agreeableness_Dad'

In [82]:
GenerateLatentSlopeString('Mom','Overlap','Agreeableness')

'MomAgreeablenessSlope =~ -0.569*W5Agreeableness_Mom+-0.364*W7Agreeableness_Mom+-0.059*W10Agreeableness_Mom+0.13*W11Agreeableness_Mom+0.274*W12Agreeableness_Mom+0.569*W13Agreeableness_Mom'

In [87]:
GenerateLatentSlopeString('Dad','Overlap','Agreeableness')

'DadAgreeablenessSlope =~ -0.569*W5Agreeableness_Dad+-0.364*W7Agreeableness_Dad+0.13*W11Agreeableness_Dad+0.274*W12Agreeableness_Dad+0.569*W13Agreeableness_Dad'

In [106]:
GenerateLatentSlopeString('Dad','Overlap','Conscientiousness')

'DadConscientiousnessSlope =~ -0.569*W5Conscientiousness_Dad+-0.364*W7Conscientiousness_Dad+0.13*W11Conscientiousness_Dad+0.274*W12Conscientiousness_Dad+0.569*W13Conscientiousness_Dad'

In [95]:
GenerateLatentSlopeString('Child','Overlap','Neuroticism')

'ChildNeuroticismSlope =~ -0.569*W5Neuroticism_Child+-0.364*W7Neuroticism_Child+-0.059*W10Neuroticism_Child+0.13*W11Neuroticism_Child+0.274*W12Neuroticism_Child+0.569*W13Neuroticism_Child'

In [96]:
GenerateLatentSlopeString('Mom','Full','Neuroticism')

'MomNeuroticismSlope =~ -0.764*W1Neuroticism_Mom+-0.569*W3Neuroticism_Mom+-0.374*W5Neuroticism_Mom+-0.17*W7Neuroticism_Mom+0.136*W10Neuroticism_Mom+0.324*W11Neuroticism_Mom+0.468*W12Neuroticism_Mom+0.764*W13Neuroticism_Mom'

In [99]:
GenerateLatentSlopeString('Dad','Overlap','Neuroticism')

'DadNeuroticismSlope =~ -0.569*W5Neuroticism_Dad+-0.364*W7Neuroticism_Dad+0.13*W11Neuroticism_Dad+0.274*W12Neuroticism_Dad+0.569*W13Neuroticism_Dad'

In [104]:
GenerateLatentSlopeString('Dad','Overlap','Opennesss')

'DadOpennesssSlope =~ -0.569*W5Opennesss_Dad+-0.364*W7Opennesss_Dad+0.13*W11Opennesss_Dad+0.274*W12Opennesss_Dad+0.569*W13Opennesss_Dad'